# Collect feature data into master dataframe
---

# Import Modules

In [1]:
import os
print(os.getcwd())
import sys

import pandas as pd

# #########################################################
from methods import get_df_octa_vol, get_df_eff_ox
from methods import get_df_dft
from methods import get_df_job_ids

/home/raulf2012/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/workflow/feature_engineering


# Read feature dataframes

In [2]:
df_octa_vol = get_df_octa_vol()

df_eff_ox = get_df_eff_ox()

df_dft = get_df_dft()

df_job_ids = get_df_job_ids()

# Setting proper indices for join

In [3]:
df_eff_ox = df_eff_ox.set_index(
    ["compenv", "slab_id", "ads", "active_site", "att_num", ],
    drop=True)

df_octa_vol = df_octa_vol.set_index(
    ["compenv", "slab_id", "ads", "active_site", "att_num", ],
    drop=True)

# Combine dataframes

In [4]:
df_list = [
    df_eff_ox,
    df_octa_vol,
    ]

df_features = pd.concat(df_list, axis=1)
df_features.head()

eff_oxid_state  \
compenv slab_id     ads active_site att_num                   
nersc   fosurufu_23 o   43          1              7.333333   
                        45          1              7.333333   
        galopuba_86 o   21          1              6.000000   
        gubufafu_74 o   16          1              8.000000   
                        20          1              8.000000   

                                             active_o_metal_dist  ir_o_mean  \
compenv slab_id     ads active_site att_num                                   
nersc   fosurufu_23 o   43          1                   1.757003   1.952891   
                        45          1                   1.751524   1.952891   
        galopuba_86 o   21          1                   1.749184   6.507621   
        gubufafu_74 o   16          1                   1.799507   1.872250   
                        20          1                   1.785505   1.872250   

                                             ir_o_std  octa_vol  
compenv slab_id     ads active_site att_num                      
nersc   fosurufu_23 o   43          1        0.163121  9.310264  
                        45          1        0.163121  9.310264  
        galopuba_86 o   21          1        1.266644  9.719644  
        gubufafu_74 o   16          1        0.068363  8.732127  
                        20          1        0.068363  8.732127

# Adding in bulk data

In [5]:
def method(row_i):
    new_column_values_dict = {
        "dH_bulk": None,
        "volume_pa": None,
        }

    # #####################################################
    slab_id_i = row_i.name[1]
    # #####################################################
    bulk_ids = df_job_ids[df_job_ids.slab_id == slab_id_i].bulk_id.unique()
    mess_i = "ikjisdjf"
    assert len(bulk_ids) == 1, mess_i
    bulk_id_i = bulk_ids[0]
    # #####################################################

    # #####################################################
    row_dft_i = df_dft.loc[bulk_id_i]
    # #####################################################
    dH_i = row_dft_i.dH
    volume_pa = row_dft_i.volume_pa
    # #####################################################


    # #####################################################
    new_column_values_dict["dH_bulk"] = dH_i
    new_column_values_dict["volume_pa"] = volume_pa
    # #####################################################
    for key, value in new_column_values_dict.items():
        row_i[key] = value
    return(row_i)

df_features = df_features.apply(method, axis=1)

In [6]:
# assert False

# Save data to pickle

In [7]:
root_path_i = os.path.join(
    os.environ["PROJ_irox_oer"],
    "workflow/feature_engineering")

# Pickling data ###########################################
import os; import pickle
directory = os.path.join(root_path_i, "out_data")
if not os.path.exists(directory): os.makedirs(directory)
path_i = os.path.join(root_path_i, "out_data/df_features.pickle")
with open(path_i, "wb") as fle:
    pickle.dump(df_features, fle)
# #########################################################

# #########################################################
import pickle; import os
with open(path_i, "rb") as fle:
    df_features = pickle.load(fle)
# #########################################################

In [8]:
from methods import get_df_features
get_df_features()

eff_oxid_state  \
compenv slab_id     ads active_site att_num                   
nersc   fosurufu_23 o   43          1              7.333333   
                        45          1              7.333333   
        galopuba_86 o   21          1              6.000000   
        gubufafu_74 o   16          1              8.000000   
                        20          1              8.000000   
...                                                     ...   
slac    vuraruna_65 o   50          1                   NaN   
        wefakuko_75 o   39          1                   NaN   
                        40          1                   NaN   
                        42          1                   NaN   
                        43          1                   NaN   

                                             active_o_metal_dist  ir_o_mean  \
compenv slab_id     ads active_site att_num                                   
nersc   fosurufu_23 o   43          1                   1.757003   1.952891   
                        45          1                   1.751524   1.952891   
        galopuba_86 o   21          1                   1.749184   6.507621   
        gubufafu_74 o   16          1                   1.799507   1.872250   
                        20          1                   1.785505   1.872250   
...                                                          ...        ...   
slac    vuraruna_65 o   50          1                   1.732568        NaN   
        wefakuko_75 o   39          1                   1.768140        NaN   
                        40          1                   1.750359        NaN   
                        42          1                   1.763135        NaN   
                        43          1                   1.742876        NaN   

                                             ir_o_std  octa_vol   dH_bulk  \
compenv slab_id     ads active_site att_num                                 
nersc   fosurufu_23 o   43          1        0.163121  9.310264 -0.595820   
                        45          1        0.163121  9.310264 -0.595820   
        galopuba_86 o   21          1        1.266644  9.719644 -0.606853   
        gubufafu_74 o   16          1        0.068363  8.732127 -0.418717   
                        20          1        0.068363  8.732127 -0.418717   
...                                               ...       ...       ...   
slac    vuraruna_65 o   50          1             NaN       NaN  0.337350   
        wefakuko_75 o   39          1             NaN       NaN -0.580270   
                        40          1             NaN       NaN -0.580270   
                        42          1             NaN       NaN -0.580270   
                        43          1             NaN       NaN -0.580270   

                                             volume_pa  
compenv slab_id     ads active_site att_num             
nersc   fosurufu_23 o   43          1        13.422324  
                        45          1        13.422324  
        galopuba_86 o   21          1        14.047474  
        gubufafu_74 o   16          1        12.740641  
                        20          1        12.740641  
...                                                ...  
slac    vuraruna_65 o   50          1        10.874901  
        wefakuko_75 o   39          1        12.348048  
                        40          1        12.348048  
                        42          1        12.348048  
                        43          1        12.348048  

[99 rows x 7 columns]

In [9]:
# # #########################################################
# import pickle; import os
# path_i = os.path.join(
#     os.environ["PROJ_irox_oer"],
#     "workflow/feature_engineering",
#     "out_data/df_features.pickle")
# with open(path_i, "rb") as fle:
#     df_features = pickle.load(fle)
# # #########################################################

In [10]:
# # Pickling data ###########################################
# import os; import pickle
# directory = "out_data"
# if not os.path.exists(directory): os.makedirs(directory)
# with open(os.path.join(directory, "df_features.pickle"), "wb") as fle:
#     pickle.dump(df_features, fle)
# # #########################################################

In [11]:
# row_i = df_features.iloc[0]

# slab_id_i = row_i.name[1]

# bulk_ids = df_job_ids[df_job_ids.slab_id == slab_id_i].bulk_id.unique()
# mess_i = "ikjisdjf"
# assert len(bulk_ids) == 1, mess_i
# bulk_id_i = bulk_ids[0]

# row_dft_i = df_dft.loc[bulk_id_i]

# dH_i = row_dft_i.dH
# volume_pa = row_dft_i.volume_pa

In [12]:
# # def method(row_i, argument_0, optional_arg=None):
# def method(row_i):
#     new_column_values_dict = {
#         "TEMP": None,
#         }


#     # #########################################################
#     # row_i = df_dft.iloc[0]
#     # #########################################################
#     bulk_id_i = row_i.name
#     # #########################################################

#     tmp = df_job_ids[df_job_ids.bulk_id == bulk_id_i]
#     if tmp.shape[0] > 0:
#         print(bulk_id_i)
#     # print(tmp.shape[0])

#     # bulk_id_i

#     # #########################################################################
#     for key, value in new_column_values_dict.items():
#         row_i[key] = value
#     return(row_i)

# df_i = df_dft

# # arg1 = "TEMP_0"
# # df_i["column_name"] = df_i.apply(
# df_i = df_i.apply(
#     method,
#     axis=1,
#     # args=(arg1, ),
#     # optional_arg="TEMP_1"
#     )
# df_i

In [13]:
# # #########################################################
# # row_i = df_dft.iloc[0]
# row_i = df_dft.loc["b49kx4c19q"]
# # #########################################################
# bulk_id_i = row_i.name
# # #########################################################

# tmp = df_job_ids[df_job_ids.bulk_id == bulk_id_i]
# print(tmp.shape[0])

# # bulk_id_i
# tmp

In [14]:
# row_i

In [15]:
# lst_0 = list(set(df_job_ids.bulk_id.tolist()))
# lst_1 = df_dft.index.tolist()

# # a = [1,2,3,4,5]
# # b = [1,3,5,6]

# list(set(lst_0) & set(lst_1))